In [1]:
import pandas as pd
import sqlite3
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Hop teaming table
db = sqlite3.connect('data/hop_team.sqlite')

for chunk in tqdm(pd.read_csv('data/DocGraph_Hop_Teaming_2018.csv', chunksize = 100000)):
    chunk.columns = [x.lower().replace(' ', '_') for x in chunk.columns]
    chunk = chunk[(chunk['transaction_count'] >= 50) & (chunk['average_day_wait'] < 50)]
    chunk.to_sql('hop', db, if_exists = 'append', index = False)
    
db.close()

0it [00:00, ?it/s]

In [3]:
# NPPES table
db = sqlite3.connect('data/hop_team.sqlite')
for chunk in tqdm(pd.read_csv('data/npidata_pfile_20050523-20220213.csv', chunksize = 100000)):
    for counter in range(1,16):
        taxonomy_switch = chunk['Healthcare Provider Primary Taxonomy Switch_{}'.format(counter)] == 'Y'
        taxonomy_code_check = 'Healthcare Provider Taxonomy Code_{}'.format(counter)
        taxonomy_code = chunk.loc[taxonomy_switch, taxonomy_code_check]
        chunk.loc[taxonomy_switch, 'Healthcare Provider Taxonomy Code'] = taxonomy_code
    chunk = chunk[['NPI',
     'Entity Type Code',
     'Provider Organization Name (Legal Business Name)',
     'Provider Last Name (Legal Name)',
     'Provider First Name',
     'Provider Middle Name',
     'Provider Name Prefix Text',
     'Provider Name Suffix Text',
     'Provider Credential Text',
     'Provider First Line Business Practice Location Address',
     'Provider Second Line Business Practice Location Address',
     'Provider Business Practice Location Address City Name',
     'Provider Business Practice Location Address State Name',
     'Provider Business Practice Location Address Postal Code',
     'Healthcare Provider Taxonomy Code']]
    chunk.columns = [x.lower().replace(' ', '_') for x in chunk.columns]
    chunk.fillna(0)
    chunk['provider_business_practice_location_address_postal_code'] = chunk['provider_business_practice_location_address_postal_code'].astype(str)
    chunk['provider_business_practice_location_address_postal_code'] = [zip.replace(zip[:], zip[:5]) if len(zip) > 9 else zip for zip in chunk['provider_business_practice_location_address_postal_code']]
    chunk.to_sql('npi', db, if_exists = 'append', index = False)
db.close()

0it [00:00, ?it/s]

In [4]:
# Taxonomy table
db = sqlite3.connect('data/hop_team.sqlite')
for chunk in tqdm(pd.read_csv('data/nucc_taxonomy_220.csv', chunksize = 100000)):
    chunk = chunk[['Code',
                   'Grouping',
                   'Classification']]
    chunk.columns = [x.lower().replace(' ', '_') for x in chunk.columns]
    chunk.to_sql('taxonomy', db, if_exists = 'append', index = False)
db.close()

0it [00:00, ?it/s]

In [5]:
# CBSA-to-ZIP table
db = sqlite3.connect('data/hop_team.sqlite')
cbsa_excel = pd.read_excel('data/ZIP_CBSA_122021.xlsx', dtype=str)
cbsa_excel.to_sql('cbsa', db, if_exists = 'replace', index = False)
db.close()

In [12]:
#Indices for tables
db = sqlite3.connect('data/hop_team.sqlite')
db.execute('CREATE INDEX from_npi ON hop(from_npi)')
db.execute('CREATE INDEX to_npi ON hop(to_npi)')
db.execute('CREATE INDEX provider_business_practice_location_address_postal_code ON npi(provider_business_practice_location_address_postal_code)')
db.execute('CREATE INDEX provider_business_practice_location_address_city_name ON npi(provider_business_practice_location_address_city_name)')
db.execute('CREATE INDEX provider_business_practice_location_address_state_name ON npi(provider_business_practice_location_address_state_name)')


In [54]:
#Check npi table
db = sqlite3.connect('data/hop_team.sqlite')

query = "SELECT [provider_organization_name_(legal_business_name)] FROM npi LIMIT 100"

npi_sqlite = pd.read_sql(query, db)

db.close()

In [55]:
npi_sqlite

,provider_organization_name_(legal_business_name)
0,None
1,None
2,"CUMBERLAND COUNTY HOSPITAL SYSTEM, INC"
3,None
4,None
...,...
95,None
96,AMOSKEAG HEALTH
97,NOVAMED MANAGEMENT SERVICES LLC
98,None


In [ ]:
db = sqlite3.connect('data/hop_team.sqlite')

query = """
SELECT 
    h.from_npi,
    h.to_npi,
    h.patient_count,
    h.transaction_count,
    h.average_day_wait,
    h.std_day_wait,
    n1.entity_type_code AS from_npi_entity_code,
    n2.entity_type_code AS to_npi_entity_code,
    n1.healthcare_provider_taxonomy_code AS from_npi_taxonomy_code,
    n2.healthcare_provider_taxonomy_code AS to_npi_taxonomy_code,
    t1.grouping AS from_npi_taxonomy_grouping,
    t1.classification AS from_npi_taxonomy_classification,
    t2.grouping AS to_npi_taxonomy_grouping,
    t2.classification AS to_npi_taxonomy_classification
FROM hop AS h
INNER JOIN npi AS n1
ON h.from_npi = n1.npi
INNER JOIN npi AS n2
ON h.to_npi = n2.npi
INNER JOIN taxonomy AS t1
ON t1.code = n2.healthcare_provider_taxonomy_code
INNER JOIN taxonomy AS t2
ON t2.code = n2.healthcare_provider_taxonomy_code
WHERE from_npi_entity_code = 1.0
    AND to_npi_entity_code = 2.0
"""
hop_npi_taxonomy_sqlite = pd.read_sql(query, db)

db.close()

In [12]:
db = sqlite3.connect('data/hop_team.sqlite')

query = "SELECT * FROM npi WHERE provider_business_practice_location_address_state_name = 'TN' AND provider_business_practice_location_address_state_name ILIKE 'VANDERBILT'"

npi_sqlite = pd.read_sql(query, db)

db.close()

DatabaseError: Execution failed on sql 'SELECT * FROM npi WHERE provider_business_practice_location_address_state_name = 'TN' AND provider_business_practice_location_address_state_name ILIKE 'VANDERBILT'': near "ILIKE": syntax error

In [3]:
npi_sqlite

,npi,entity_type_code,provider_organization_name_(legal_business_name),provider_last_name_(legal_name),provider_first_name,provider_middle_name,provider_name_prefix_text,provider_name_suffix_text,provider_credential_text,provider_first_line_business_practice_location_address,provider_second_line_business_practice_location_address,provider_business_practice_location_address_city_name,provider_business_practice_location_address_state_name,provider_business_practice_location_address_postal_code,healthcare_provider_taxonomy_code
0,1841293891,1.0,None,GIBBS,ELMER,RICKEY,DR.,None,M.D.,49 CLEVELAND ST 310,None,CROSSVILLE,TN,38555,208600000X
1,1659374601,1.0,None,OBERDICK,WENDY,TIPTON,None,None,MD,105 W STONE DR,STE 1F,KINGSPORT,TN,37660,207Q00000X
2,1134122187,1.0,None,RUDNICKE,CHERYL,DENISE,MRS.,None,CRNP,250 25TH AVE N,STE 412,NASHVILLE,TN,37203,363L00000X
3,1003819046,1.0,None,NYLANDER,BARBARA,H,None,None,M.D.,345 23RD AVE N,SUITE 209,NASHVILLE,TN,37203,207VG0400X
4,1750384780,1.0,None,PERRIGIN,JULIE,A,DR.,None,MD,219 CHURCH ST,None,DICKSON,TN,37055,207Q00000X
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123505,1821743493,1.0,None,RUSSELL,KILA,None,None,None,None,3818 S LEE HWY,None,MC DONALD,TN,373535876,224P00000X
123506,1184379760,1.0,None,GAYLE,KAYLA,DANIELLE,None,None,FNP,2325 CRESTMOOR RD STE 101,None,NASHVILLE,TN,372152032,363LF0000X
123507,1891440475,1.0,None,JORDY,JEFFREY,LEE,None,None,LCSW,210 30TH AVE N APT 301,None,NASHVILLE,TN,372031583,101YM0800X
123508,1346995917,1.0,None,TAYLOR,KERI,None,None,None,OTR/L,2018 EXETER RD,None,GERMANTOWN,TN,381383945,225X00000X


In [5]:
db = sqlite3.connect('data/hop_team.sqlite')

query = "SELECT * FROM taxonomy"

taxonomy_sqlite = pd.read_sql(query, db)

db.close()

In [6]:
taxonomy_sqlite

,code,grouping,classification
0,193200000X,Group,Multi-Specialty
1,193400000X,Group,Single Specialty
2,207K00000X,Allopathic & Osteopathic Physicians,Allergy & Immunology
3,207KA0200X,Allopathic & Osteopathic Physicians,Allergy & Immunology
4,207KI0005X,Allopathic & Osteopathic Physicians,Allergy & Immunology
...,...,...,...
863,343800000X,Transportation Services,Secured Medical Transport (VAN)
864,344600000X,Transportation Services,Taxi
865,347D00000X,Transportation Services,Train
866,347E00000X,Transportation Services,Transportation Broker


In [204]:
#Hospitals in Nashville metro and surrouding areas (cbsa) 
db = sqlite3.connect('data/hop_team.sqlite')

query = """
SELECT to_npi,  
       [provider_organization_name_(legal_business_name)],
       provider_first_line_business_practice_location_address,
       provider_business_practice_location_address_city_name,
       code,
       grouping,
       classification,
       SUM(transaction_count)
FROM hop 
INNER JOIN npi
ON hop.to_npi = npi.npi
INNER JOIN taxonomy
ON taxonomy.code=npi.healthcare_provider_taxonomy_code
INNER JOIN cbsa
ON cbsa.zip = npi.provider_business_practice_location_address_postal_code
WHERE provider_business_practice_location_address_city_name = 'NASHVILLE' AND 
  provider_business_practice_location_address_state_name = 'TN' AND
  grouping = 'Hospitals'
GROUP BY to_npi
ORDER BY SUM(transaction_count) DESC
LIMIT 100

"""

hop20 = pd.read_sql(query, db)

db.close()

In [205]:
pd.options.display.max_rows=100
hop20

,to_npi,provider_organization_name_(legal_business_name),provider_first_line_business_practice_location_address,provider_business_practice_location_address_city_name,code,grouping,classification,SUM(transaction_count)
0,1396882205,VANDERBILT UNIVERSITY MEDICAL CENTER,1211 MEDICAL CENTER DRIVE,NASHVILLE,282N00000X,Hospitals,General Acute Care Hospital,1263851
1,1023055126,"HCA HEALTH SERVICES OF TENNESSEE, INC.",2300 PATTERSON ST,NASHVILLE,282N00000X,Hospitals,General Acute Care Hospital,587886
2,1780778969,SAINT THOMAS WEST HOSPITAL,2000 CHURCH ST,NASHVILLE,282N00000X,Hospitals,General Acute Care Hospital,288411
3,1720032345,"HCA HEALTH SERVICES OF TENNESSEE, INC.",391 WALLACE RD,NASHVILLE,282N00000X,Hospitals,General Acute Care Hospital,123901
4,1932146032,"HCA HEALTH SERVICES OF TENNESSEE, INC.",2300 PATTERSON ST,NASHVILLE,282N00000X,Hospitals,General Acute Care Hospital,22550
5,1568551455,NASHVILLE GENERAL HOSPITAL,1818 ALBION ST,NASHVILLE,282N00000X,Hospitals,General Acute Care Hospital,21248
6,1215937982,SELECT SPECIALTY HOSPITAL - NASHVILLE LLC,2000 HAYES STREET,NASHVILLE,282E00000X,Hospitals,Long Term Care Hospital,14972
7,1598738205,VANDERBILT STALLWORTH REHABILITATION HOSPITAL LP,2201 CHILDRENS WAY,NASHVILLE,283X00000X,Hospitals,Rehabilitation Hospital,4517
8,1093769606,HTI MEMORIAL HOSPITAL CORPORATION,3441 DICKERSON PIKE,NASHVILLE,282N00000X,Hospitals,General Acute Care Hospital,3690
9,1124065255,"HCA HEALTH SERVICES OF TENNESSEE, INC.",391 WALLACE RD,NASHVILLE,282N00000X,Hospitals,General Acute Care Hospital,3106


In [225]:
#Habeeb main query
db = sqlite3.connect('data/hop_team.sqlite')

query = """
SELECT 
    h.from_npi,
    h.to_npi,
    h.patient_count,
    h.transaction_count,
    h.average_day_wait,
    n.provider_first_line_business_practice_location_address AS address,
    n.[provider_organization_name_(legal_business_name)] AS to_npi_name,
    n.provider_business_practice_location_address_city_name AS to_npi_city,
    n.provider_business_practice_location_address_state_name AS state,
    t.grouping AS to_npi_grouping,
    t.classification AS to_npi_classification
FROM hop AS h
INNER JOIN npi AS n
ON h.to_npi = n.npi
INNER JOIN taxonomy AS t
ON t.code = n.healthcare_provider_taxonomy_code
WHERE n.entity_type_code = '2'
    AND to_npi_city = 'NASHVILLE'
    AND state = 'TN'
"""
nashville_cbsa = pd.read_sql(query, db)

db.close()

In [227]:
nashville_cbsa[nashville_cbsa['to_npi_name'] LIKE 'VANDERBILT UNIVERSITY MEDICAL CENTER']


SyntaxError: invalid syntax (<ipython-input-227-a1855bae8134>, line 1)

In [226]:
nashville_cbsa

,from_npi,provider_organization_name_(legal_business_name),to_npi,patient_count,transaction_count,average_day_wait,address,to_npi_name,to_npi_city,state,to_npi_grouping,to_npi_classification
0,1548284060,"CARIS HEALTHCARE, LP",1609879956,31,55,37.200,2525 PERIMETER PLACE DR,"CARIS HEALTHCARE, LP",NASHVILLE,TN,Agencies,"Hospice Care, Community Based"
1,1780040139,"CARIS HEALTHCARE, LP",1609879956,30,51,8.863,2525 PERIMETER PLACE DR,"CARIS HEALTHCARE, LP",NASHVILLE,TN,Agencies,"Hospice Care, Community Based"
2,1558386433,"CARIS HEALTHCARE, LP",1609879956,20,57,11.930,2525 PERIMETER PLACE DR,"CARIS HEALTHCARE, LP",NASHVILLE,TN,Agencies,"Hospice Care, Community Based"
3,1417951302,"CARIS HEALTHCARE, LP",1609879956,17,59,0.814,2525 PERIMETER PLACE DR,"CARIS HEALTHCARE, LP",NASHVILLE,TN,Agencies,"Hospice Care, Community Based"
4,1437429552,"CARIS HEALTHCARE, LP",1609879956,67,87,25.161,2525 PERIMETER PLACE DR,"CARIS HEALTHCARE, LP",NASHVILLE,TN,Agencies,"Hospice Care, Community Based"
...,...,...,...,...,...,...,...,...,...,...,...,...
66760,1861478489,VICTOR KHA DO PLLC,1477042588,64,82,9.841,397 WALLACE RD STE 415,VICTOR KHA DO PLLC,NASHVILLE,TN,Allopathic & Osteopathic Physicians,Internal Medicine
66761,1114961513,VICTOR KHA DO PLLC,1477042588,50,64,22.891,397 WALLACE RD STE 415,VICTOR KHA DO PLLC,NASHVILLE,TN,Allopathic & Osteopathic Physicians,Internal Medicine
66762,1245393057,VICTOR KHA DO PLLC,1477042588,59,79,10.785,397 WALLACE RD STE 415,VICTOR KHA DO PLLC,NASHVILLE,TN,Allopathic & Osteopathic Physicians,Internal Medicine
66763,1396003356,VICTOR KHA DO PLLC,1477042588,74,131,0.000,397 WALLACE RD STE 415,VICTOR KHA DO PLLC,NASHVILLE,TN,Allopathic & Osteopathic Physicians,Internal Medicine


In [186]:
#Vandy midtown referral types
db = sqlite3.connect('data/hop_team.sqlite')

query = """
SELECT classification,
       SUM(transaction_count) AS Vandy_Mid_Referral,
       ROUND(SUM(transaction_count) * 100.0 /SUM(SUM(transaction_count)) OVER (),2) AS Percent_of_Total_referrals
FROM hop 
INNER JOIN npi
ON hop.from_npi = npi.npi
INNER JOIN taxonomy
ON taxonomy.code=npi.healthcare_provider_taxonomy_code
INNER JOIN cbsa
ON cbsa.zip = npi.provider_business_practice_location_address_postal_code
WHERE to_npi ='1396882205'
GROUP BY classification
ORDER BY SUM(transaction_count) DESC
"""

vandy_mid = pd.read_sql(query, db)

db.close()

In [187]:
vandy_mid

,classification,Vandy_Mid_Referral,Percent_of_Total_referrals
0,Internal Medicine,286366,34.00
1,Radiology,105363,12.51
2,General Acute Care Hospital,53179,6.31
3,Family Medicine,52619,6.25
4,Nurse Practitioner,48838,5.80
5,Emergency Medicine,26015,3.09
6,Durable Medical Equipment & Medical Supplies,21596,2.56
7,Psychiatry & Neurology,20126,2.39
8,Ophthalmology,18614,2.21
9,Clinic/Center,18026,2.14


In [229]:
#Vamsi code with NANs for vandy 
db = sqlite3.connect('data/hop_team.sqlite')

query = '''
WITH vandy AS (
    SELECT
        classification,
        [provider_organization_name_(legal_business_name)],
        AVG(patient_count) AS avg_vandy,
        SUM(CAST(patient_count AS INT)) AS sum_vandy
    FROM hop
    LEFT JOIN npi
    ON to_npi = npi
    LEFT JOIN taxonomy
    ON healthcare_provider_taxonomy_code = code
    WHERE entity_type_code = 2.0 AND provider_business_practice_location_address_city_name = 'NASHVILLE' AND [provider_organization_name_(legal_business_name)] LIKE '%VANDERBILT%'
    GROUP BY classification, [provider_organization_name_(legal_business_name)]
    ORDER BY AVG(patient_count) DESC
),
overall AS (
    SELECT
        classification,
        AVG(patient_count) AS overall_avg,
        CAST(SUM(patient_count) AS INT) AS overall_sum
    FROM hop
    LEFT JOIN npi
    ON to_npi = npi
    LEFT JOIN taxonomy
    ON healthcare_provider_taxonomy_code = code
    WHERE entity_type_code = 2.0 AND provider_business_practice_location_address_city_name = 'NASHVILLE' AND provider_business_practice_location_address_state_name = 'TN'
    GROUP BY classification
    ORDER BY AVG(patient_count) DESC
)
SELECT
    overall.classification,
    overall_avg,
    avg_vandy,
    sum_vandy,
    overall_sum
FROM overall
LEFT JOIN vandy
ON vandy.classification = overall.classification
GROUP BY overall.classification
ORDER BY avg_vandy DESC
'''
s = pd.read_sql(query, db)

db.close()

In [230]:
s

,classification,overall_avg,avg_vandy,sum_vandy,overall_sum
0,Radiology,304.316199,235.408451,16714.0,1213613
1,Internal Medicine,212.894482,228.814437,1166496.0,3950470
2,General Acute Care Hospital,237.801403,209.721938,779117.0,1830833
3,Home Health,102.145882,177.583333,2131.0,43412
4,Rehabilitation Hospital,101.119048,101.119048,4247.0,4247
5,Pharmacy,67.841689,80.888235,13751.0,25712
6,Clinic/Center,137.485431,55.652174,1280.0,235925
7,Urology,224.541985,NaN,NaN,176490
8,Thoracic Surgery (Cardiothoracic Vascular Surg...,116.860465,NaN,NaN,5025
9,Surgery,156.914793,NaN,NaN,132593


In [195]:
#Habeeb main query
db = sqlite3.connect('data/hop_team.sqlite')

query = """
SELECT 
    h.from_npi,
    h.to_npi,
    h.patient_count,
    h.transaction_count,
    h.average_day_wait,
    n.provider_first_line_business_practice_location_address AS address,
    n.[provider_organization_name_(legal_business_name)] AS to_npi_name,
    n.provider_business_practice_location_address_city_name AS to_npi_city,
    n.provider_business_practice_location_address_state_name AS state,
    t.grouping AS to_npi_grouping,
    t.classification AS to_npi_classification
FROM hop AS h
INNER JOIN npi AS n
ON h.to_npi = n.npi
INNER JOIN taxonomy AS t
ON t.code = n.healthcare_provider_taxonomy_code
WHERE n.entity_type_code = '2'
    AND to_npi_city = 'NASHVILLE'
    AND state = 'TN'
"""
nashville_cbsa = pd.read_sql(query, db)

db.close()

In [196]:
nashville_cbsa[nashville_cbsa['to_npi_name'] == 'VANDERBILT UNIVERSITY MEDICAL CENTER']


,from_npi,to_npi,patient_count,transaction_count,average_day_wait,address,to_npi_name,to_npi_city,state,to_npi_grouping,to_npi_classification
46697,1518945195,1558408633,648,1577,0.682,1601 23RD AVE S,VANDERBILT UNIVERSITY MEDICAL CENTER,NASHVILLE,TN,Hospitals,General Acute Care Hospital
46698,1528034675,1558408633,69,76,5.237,1601 23RD AVE S,VANDERBILT UNIVERSITY MEDICAL CENTER,NASHVILLE,TN,Hospitals,General Acute Care Hospital
46699,1538153937,1558408633,91,124,45.532,1601 23RD AVE S,VANDERBILT UNIVERSITY MEDICAL CENTER,NASHVILLE,TN,Hospitals,General Acute Care Hospital
46700,1538243639,1558408633,75,193,1.472,1601 23RD AVE S,VANDERBILT UNIVERSITY MEDICAL CENTER,NASHVILLE,TN,Hospitals,General Acute Care Hospital
46701,1548284060,1558408633,127,189,40.582,1601 23RD AVE S,VANDERBILT UNIVERSITY MEDICAL CENTER,NASHVILLE,TN,Hospitals,General Acute Care Hospital
...,...,...,...,...,...,...,...,...,...,...,...
66657,1669554556,1972027159,30,75,23.080,2906 FOSTER CREIGHTON DR STE 100,VANDERBILT UNIVERSITY MEDICAL CENTER,NASHVILLE,TN,Ambulatory Health Care Facilities,Clinic/Center
66658,1104202761,1972027159,71,332,2.190,2906 FOSTER CREIGHTON DR STE 100,VANDERBILT UNIVERSITY MEDICAL CENTER,NASHVILLE,TN,Ambulatory Health Care Facilities,Clinic/Center
66659,1124024922,1972027159,41,85,41.929,2906 FOSTER CREIGHTON DR STE 100,VANDERBILT UNIVERSITY MEDICAL CENTER,NASHVILLE,TN,Ambulatory Health Care Facilities,Clinic/Center
66660,1992897326,1972027159,19,51,17.588,2906 FOSTER CREIGHTON DR STE 100,VANDERBILT UNIVERSITY MEDICAL CENTER,NASHVILLE,TN,Ambulatory Health Care Facilities,Clinic/Center


In [155]:
#TriStar Centennial referral types
db = sqlite3.connect('data/hop_team.sqlite')

query = """
SELECT classification,
       SUM(transaction_count) AS TriStar_Centennial_Referral,
       ROUND(SUM(transaction_count) * 100.0 /SUM(SUM(transaction_count)) OVER (),2) AS Percent_of_Total_referrals
FROM hop 
INNER JOIN npi
ON hop.from_npi = npi.npi
INNER JOIN taxonomy
ON taxonomy.code=npi.healthcare_provider_taxonomy_code
INNER JOIN cbsa
ON cbsa.zip = npi.provider_business_practice_location_address_postal_code
WHERE to_npi ='1023055126'
GROUP BY classification
ORDER BY SUM(transaction_count) DESC
"""

tristar_centennial = pd.read_sql(query, db)

db.close()

In [157]:
tristar_centennial

,classification,TriStar_Centennial_Referral,Percent_of_Total_referrals
0,Internal Medicine,135192,37.38
1,Radiology,60877,16.83
2,Pathology,25500,7.05
3,Emergency Medicine,22298,6.17
4,General Acute Care Hospital,13023,3.60
5,Orthopaedic Surgery,12366,3.42
6,Family Medicine,10770,2.98
7,Nurse Practitioner,10437,2.89
8,Psychiatry & Neurology,8753,2.42
9,Surgery,7436,2.06


In [156]:
#St.Thomas West referral types
db = sqlite3.connect('data/hop_team.sqlite')

query = """
SELECT classification,
       SUM(transaction_count) AS St_Thomas_West_Referrals,
       ROUND(SUM(transaction_count) * 100.0 /SUM(SUM(transaction_count)) OVER (),2) AS Percent_of_Total_referrals
FROM hop 
INNER JOIN npi
ON hop.from_npi = npi.npi
INNER JOIN taxonomy
ON taxonomy.code=npi.healthcare_provider_taxonomy_code
INNER JOIN cbsa
ON cbsa.zip = npi.provider_business_practice_location_address_postal_code
WHERE to_npi ='1780778969'
GROUP BY classification
ORDER BY SUM(transaction_count) DESC

"""

st_thomas_west = pd.read_sql(query, db)

db.close()

In [158]:
st_thomas_west

,classification,St_Thomas_West_Referrals,Percent_of_Total_referrals
0,Internal Medicine,109899,51.91
1,Radiology,26920,12.72
2,Pathology,8719,4.12
3,Surgery,7959,3.76
4,Orthopaedic Surgery,5881,2.78
5,General Acute Care Hospital,5690,2.69
6,Emergency Medicine,5091,2.40
7,Hospitalist,4976,2.35
8,"Nurse Anesthetist, Certified Registered",4436,2.10
9,Anesthesiology,3683,1.74


In [159]:
#St Thomas Rutherford referral types
db = sqlite3.connect('data/hop_team.sqlite')

query = """
SELECT classification,
       SUM(transaction_count),
       ROUND(SUM(transaction_count) * 100.0 /SUM(SUM(transaction_count)) OVER (),2) AS Percent_of_Total_referrals
FROM hop 
INNER JOIN npi
ON hop.from_npi = npi.npi
INNER JOIN taxonomy
ON taxonomy.code=npi.healthcare_provider_taxonomy_code
INNER JOIN cbsa
ON cbsa.zip = npi.provider_business_practice_location_address_postal_code
WHERE to_npi ='1164590386'
GROUP BY classification
ORDER BY SUM(transaction_count) DESC

"""

st_thomas_rutherford = pd.read_sql(query, db)

db.close()

In [160]:
st_thomas_rutherford

,classification,SUM(transaction_count),Percent_of_Total_referrals
0,Internal Medicine,80477,40.24
1,Radiology,35389,17.69
2,Family Medicine,12175,6.09
3,Ambulance,8385,4.19
4,Emergency Medicine,8352,4.18
5,Anesthesiology,7848,3.92
6,Specialist,7590,3.79
7,General Acute Care Hospital,7362,3.68
8,Orthopaedic Surgery,4391,2.20
9,Nurse Practitioner,4126,2.06


In [167]:
#TriStar summit referral types
db = sqlite3.connect('data/hop_team.sqlite')

query = """
SELECT classification,
       SUM(transaction_count) AS TriStar_Summit_Referrals,
       ROUND(SUM(transaction_count) * 100.0 /SUM(SUM(transaction_count)) OVER (),2) AS Percent_of_Total_referrals
FROM hop 
INNER JOIN npi
ON hop.from_npi = npi.npi
INNER JOIN taxonomy
ON taxonomy.code=npi.healthcare_provider_taxonomy_code
INNER JOIN cbsa
ON cbsa.zip = npi.provider_business_practice_location_address_postal_code
WHERE to_npi ='1982650024'
GROUP BY classification
ORDER BY SUM(transaction_count) DESC

"""

tristar_summit = pd.read_sql(query, db)

db.close()

In [168]:
tristar_summit

,classification,TriStar_Summit_Referrals,Percent_of_Total_referrals
0,Internal Medicine,70904,42.62
1,Radiology,38094,22.90
2,Emergency Medicine,7437,4.47
3,Family Medicine,6062,3.64
4,Physical Medicine & Rehabilitation,3977,2.39
5,Nurse Practitioner,3853,2.32
6,General Acute Care Hospital,3659,2.20
7,Specialist,3170,1.91
8,Pathology,3136,1.89
9,Ambulance,2916,1.75


In [177]:
db = sqlite3.connect('data/hop_team.sqlite')

query = """
WITH A AS(
SELECT classification,
       SUM(transaction_count) AS TriStar_Summit_Referrals,
       ROUND(SUM(transaction_count) * 100.0 /SUM(SUM(transaction_count)) OVER (),2) AS Percent_of_Total_referrals
FROM hop 
INNER JOIN npi
ON hop.from_npi = npi.npi
INNER JOIN taxonomy
ON taxonomy.code=npi.healthcare_provider_taxonomy_code
INNER JOIN cbsa
ON cbsa.zip = npi.provider_business_practice_location_address_postal_code
WHERE to_npi ='1982650024'
GROUP BY classification
ORDER BY SUM(transaction_count) DESC
),

B AS (
SELECT classification,
       SUM(transaction_count) AS St_Thomas_Rutherford_Referrals,
       ROUND(SUM(transaction_count) * 100.0 /SUM(SUM(transaction_count)) OVER (),2) AS Percent_of_Total_referrals
FROM hop 
INNER JOIN npi
ON hop.from_npi = npi.npi
INNER JOIN taxonomy
ON taxonomy.code=npi.healthcare_provider_taxonomy_code
INNER JOIN cbsa
ON cbsa.zip = npi.provider_business_practice_location_address_postal_code
WHERE to_npi ='1164590386'
GROUP BY classification
ORDER BY SUM(transaction_count) DESC),

C AS (
SELECT classification,
       SUM(transaction_count) AS Vandy_Mid_Referral,
       ROUND(SUM(transaction_count) * 100.0 /SUM(SUM(transaction_count)) OVER (),2) AS Percent_of_Total_referrals
FROM hop 
INNER JOIN npi
ON hop.from_npi = npi.npi
INNER JOIN taxonomy
ON taxonomy.code=npi.healthcare_provider_taxonomy_code
INNER JOIN cbsa
ON cbsa.zip = npi.provider_business_practice_location_address_postal_code
WHERE to_npi ='1396882205'
GROUP BY classification
ORDER BY SUM(transaction_count) DESC),

D AS (
SELECT classification,
       SUM(transaction_count) AS TriStar_Centennial_Referral,
       ROUND(SUM(transaction_count) * 100.0 /SUM(SUM(transaction_count)) OVER (),2) AS Percent_of_Total_referrals
FROM hop 
INNER JOIN npi
ON hop.from_npi = npi.npi
INNER JOIN taxonomy
ON taxonomy.code=npi.healthcare_provider_taxonomy_code
INNER JOIN cbsa
ON cbsa.zip = npi.provider_business_practice_location_address_postal_code
WHERE to_npi ='1023055126'
GROUP BY classification
ORDER BY SUM(transaction_count) DESC)

SELECT *
FROM A
JOIN B
USING(classification)
JOIN C
USING(classification)
JOIN D
USING(classification)

"""

TEST = pd.read_sql(query, db)

db.close()

In [178]:
TEST

,classification,TriStar_Summit_Referrals,Percent_of_Total_referrals,St_Thomas_Rutherford_Referrals,Percent_of_Total_referrals,Vandy_Mid_Referral,Percent_of_Total_referrals,TriStar_Centennial_Referral,Percent_of_Total_referrals
0,Internal Medicine,70904,42.62,80477,40.24,286366,34.00,135192,37.38
1,Radiology,38094,22.90,35389,17.69,105363,12.51,60877,16.83
2,Emergency Medicine,7437,4.47,8352,4.18,26015,3.09,22298,6.17
3,Family Medicine,6062,3.64,12175,6.09,52619,6.25,10770,2.98
4,Physical Medicine & Rehabilitation,3977,2.39,1286,0.64,3661,0.43,1333,0.37
5,Nurse Practitioner,3853,2.32,4126,2.06,48838,5.80,10437,2.89
6,General Acute Care Hospital,3659,2.20,7362,3.68,53179,6.31,13023,3.60
7,Specialist,3170,1.91,7590,3.79,13263,1.57,2927,0.81
8,Pathology,3136,1.89,1511,0.76,13376,1.59,25500,7.05
9,Ambulance,2916,1.75,8385,4.19,8099,0.96,5154,1.43
